In [1]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD

In [2]:
os.environ["KAGGLE_USERNAME"] = "alinakhan11"
os.environ["KAGGLE_KEY"] = "a8f3ddee7ebb50a2559441703e9df95"

In [3]:
kg.api.authenticate()

In [4]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)

Dataset URL: https://www.kaggle.com/datasets/medahmedkrichen/devanagari-handwritten-character-datase


In [5]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})        

In [6]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [7]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [8]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,5
78196,dataset/DevanagariHandwrittenCharacterDataset/...,5
78197,dataset/DevanagariHandwrittenCharacterDataset/...,5
78198,dataset/DevanagariHandwrittenCharacterDataset/...,5


In [9]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,5
13796,dataset/DevanagariHandwrittenCharacterDataset/...,5
13797,dataset/DevanagariHandwrittenCharacterDataset/...,5
13798,dataset/DevanagariHandwrittenCharacterDataset/...,5


In [10]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [11]:
character2int

{'ra': 0,
 'chha': 1,
 'patalosaw': 2,
 'dha': 3,
 'la': 4,
 'ga': 5,
 'yaw': 6,
 'na': 7,
 '3': 8,
 'petchiryakha': 9,
 'pha': 10,
 'gya': 11,
 'da': 12,
 'kha': 13,
 'tha': 14,
 '1': 15,
 'ja': 16,
 '4': 17,
 'ma': 18,
 'motosaw': 19,
 'chhya': 20,
 'ha': 21,
 '6': 22,
 'taamatar': 23,
 '2': 24,
 'yna': 25,
 'adna': 26,
 'tra': 27,
 'kna': 28,
 'tabala': 29,
 'ba': 30,
 'waw': 31,
 '7': 32,
 'pa': 33,
 '0': 34,
 'jha': 35,
 'bha': 36,
 'daa': 37,
 '9': 38,
 '8': 39,
 'dhaa': 40,
 'gha': 41,
 'cha': 42,
 'thaa': 43,
 'ka': 44,
 '5': 45}

In [12]:
training_data["label"].replace(character2int, inplace=True)

/tmp/ipykernel_500/80905328.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training_data["label"].replace(character2int, inplace=True)
/tmp/ipykernel_500/80905328.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data["label"].replace(character2int, inplace=True)


In [13]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,45
78196,dataset/DevanagariHandwrittenCharacterDataset/...,45
78197,dataset/DevanagariHandwrittenCharacterDataset/...,45
78198,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [14]:
testing_data["label"].replace(character2int, inplace=True)

/tmp/ipykernel_500/882846922.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  testing_data["label"].replace(character2int, inplace=True)
/tmp/ipykernel_500/882846922.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_data["label"].replace(character2int, inplace=True)


In [15]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,45
13796,dataset/DevanagariHandwrittenCharacterDataset/...,45
13797,dataset/DevanagariHandwrittenCharacterDataset/...,45
13798,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [16]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=46)

In [17]:
Y_true_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [18]:
Y_true_test = to_categorical(y=testing_data["label"],num_classes=46)

In [19]:
Y_true_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [20]:
def multiclass_dnn():

    input_to_dnn = Input(shape=(1024,))
    first_dense_out = Dense(units=1024,activation="relu")(input_to_dnn)
    second_dense_out = Dense(units=1024,activation="relu")(first_dense_out)
    second_dense_out = BatchNormalization()(second_dense_out)
    output = Dense(units=46,activation="softmax")(second_dense_out)

    return Model(inputs=[input_to_dnn],outputs=[output])


In [21]:
def custom_data_generator(data_df, Y_true, mb_size):

    for time_step in range(data_df.shape[0]//mb_size):
        X_mb = list()

        for img_path in data_df.iloc[time_step*mb_size:(time_step+1)*mb_size,0]:

            img_np_array = plt.imread(img_path)
            reshaped_np_array = img_np_array.reshape(1024,)
            X_mb.append(reshaped_np_array)

        X_mb = np.array(X_mb)
        Y_true_mb = Y_true[time_step*mb_size:(time_step+1)*mb_size]

        yield X_mb, Y_true_mb

In [22]:
epochs = 50
training_data_mb_size = 782
testing_data_mb_size = 138

In [24]:

model = multiclass_dnn()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024)]            0         
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dense_2 (Dense)             (None, 46)                47150     
                                                                 
Total params: 2150446 (8.20 MB)
Trainable params: 2148398 (8.20 MB)
Non-trainable params: 2048 (8.00 KB)
______________________

In [25]:
def loss_fn(Y_true_mb,Y_pred_mb):

    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y_true_mb,
                                                                          y_pred=Y_pred_mb))

optimizer = SGD()

In [26]:
@tf.function
def training_step(X_train_mb,Y_true_train_mb):

    with tf.GradientTape() as tape:
            
        Y_pred_train_mb = model(X_train_mb, training=True)
        training_loss = loss_fn(Y_true_train_mb, Y_pred_train_mb)

    grads = tape.gradient(training_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(Y_true_train_mb,Y_pred_train_mb)

    return training_loss

In [27]:
@tf.function
def testing_forward_pass(X_test_mb,Y_true_test_mb):

    Y_pred_test_mb = model(X_test_mb,training=False)
    testing_loss = loss_fn(Y_true_test_mb,Y_pred_test_mb)
    test_acc_metric.update_state(Y_true_test_mb,Y_pred_test_mb)

    return testing_loss

In [28]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

for epoch in range(epochs):

    training_data_generator = custom_data_generator(training_data,Y_true_train,782)

    for time_step, (X_train_mb, Y_true_train_mb) in enumerate(training_data_generator):
        training_loss = training_step(X_train_mb,Y_true_train_mb)

        if (time_step+1) % 50 == 0:
            print("Epoch %d, Time Step %d, Training loss for one mini batch: %.4f"
            % (epoch+1, time_step+1, float(training_loss)))
            
    training_acc = train_acc_metric.result()    
    print("Epoch %d, Training Accuracy: %.2f" % (epoch+1,float(training_acc)))
    train_acc_metric.reset_states()

    testing_data_generator = custom_data_generator(testing_data,Y_true_test,testing_data_mb_size)

    for X_test_mb, Y_true_test_mb in testing_data_generator:
        testing_loss = testing_forward_pass(X_test_mb,Y_true_test_mb)

    print("\nEpoch %d, Testing Loss for last mini batch: %.4f" % (epoch+1,float(testing_loss)))
    testing_acc = test_acc_metric.result()
    print("Epoch %d, Testing Accuracy: %.2f" % (epoch+1,float(testing_acc)))
    test_acc_metric.reset_states()

    print("\n\n")


Epoch 1, Time Step 50, Training loss for one mini batch: 4.3820
Epoch 1, Time Step 100, Training loss for one mini batch: 4.2736
Epoch 1, Training Accuracy: 0.04

Epoch 1, Testing Loss for last mini batch: 3.8806
Epoch 1, Testing Accuracy: 0.12



Epoch 2, Time Step 50, Training loss for one mini batch: 4.2337
Epoch 2, Time Step 100, Training loss for one mini batch: 4.1966
Epoch 2, Training Accuracy: 0.07

Epoch 2, Testing Loss for last mini batch: 3.7252
Epoch 2, Testing Accuracy: 0.19



Epoch 3, Time Step 50, Training loss for one mini batch: 4.1773
Epoch 3, Time Step 100, Training loss for one mini batch: 4.1566
Epoch 3, Training Accuracy: 0.10

Epoch 3, Testing Loss for last mini batch: 3.5458
Epoch 3, Testing Accuracy: 0.23



Epoch 4, Time Step 50, Training loss for one mini batch: 4.1447
Epoch 4, Time Step 100, Training loss for one mini batch: 4.1293
Epoch 4, Training Accuracy: 0.12

Epoch 4, Testing Loss for last mini batch: 3.3824
Epoch 4, Testing Accuracy: 0.27



Epoch 5,